In [1]:
#keras Functional Model

from keras.layers import Input , Dense
from keras.models import Model

Using TensorFlow backend.


In [2]:
inputs = Input(shape=(784 , ))

In [3]:
x = Dense(64 , activation='relu')(inputs)
x = Dense(64 , activation='relu')(x)

predictions = Dense(10 , activation='softmax')(x)

In [4]:
model = Model(inputs=inputs , outputs=predictions)

model.compile(optimizer='rmsprop',
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])

model.fit(data , labels)

NameError: name 'data' is not defined

In [6]:
#模型类似函数 可以作为运算的操作
will_input_data = Input(shape=(784 , ))
y = model(will_input_data)
#模型可以被作为层一样
#调用一个模型时 即重用结构 也重用权重

In [7]:
from keras.layers import TimeDistributed

input_sequences = Input(shape=(20,784))

processed_sequences = TimeDistributed(model)(input_sequences)

In [ ]:
#http://keras-cn.readthedocs.io/en/latest/getting_started/functional_API/

http://keras-cn.readthedocs.io/en/ltest/images/multi-input-multi-output-graph.png

In [4]:
#multiple input and output
from keras.layers import Input , Embedding , LSTM , Dense
from keras.models import Model
import keras

#100 one-hot vectors(10000-dimensional)
main_input = Input(shape=(100,) , dtype='int32' ,
                  name = 'main_input')

#one_hot to 512-dimensional vectors (embedding vector)
#word2vec
x = Embedding(output_dim = 512 , input_dim = 10000,
             input_length = 100)(main_input)

#transform the vector sequence into a single vector(32-dimensional)
lstm_out = LSTM(32)(x)


#extra loss (for smoothly train LSTM Embedding layer)
auxiliary_output = Dense(1 , activation='sigmoid',
                        name='aux_output')(lstm_out)


#extra input
#auxiliary_input is unknown
auxiliary_input = Input(shape=(5,), name='aux_input')
x = keras.layers.concatenate([lstm_out , auxiliary_input])

x = Dense(64 , activation='relu')(x)
x = Dense(64 , activation='relu')(x)
x = Dense(64 , activation='relu')(x)

#main loss
main_output = Dense(1 , activation='sigmoid' ,
                   name='main_output')(x)

model = Model(inputs=[main_input , auxiliary_input],
             outputs=[main_output , auxiliary_output])


#compile method 1
model.compile(optimizer='rmsprop',
             loss=['binary_crossentropy','binary_crossentropy'],
             loss_weights=[1.0 , 0.2])

model.fit([headline_data , additional_data] , [labels , labels] , epochs=50,
         batch_size=32)

#compile moethod 2
#前提是必须人为指定name参数
model.compile(optimizer='rmsprop',
             loss={'main_output':'binary_crossentropy',
                   'aux_output':'binary_crossentropy'},
             loss_weights={'main_output':1.0,
                           'aux_output':0.2})

model.fit({'main_input':headline_data,
           'aux_input':additional_data},
         {'main_output':labels,'aux_outpus':labels},
         epoch=50 , batch_size=32)

NameError: name 'headline_data' is not defined

In [ ]:
#shared layer
import keras
from keras.layers import Input , LSTM , Dense
from keras.models import Model

#140 one-hot vectors(256-dimensional)
tweet_a = Input(shape=(140 , 256))
tweet_b = Input(shape=(140 , 256))

#shared lstm layer
shared_lstm = LSTM(64)

encoded_a = shared_lstm(tweet_a)
encoded_b = shared_lstm(tweet_b)

#concat them
merged_vector = keras.layers.concatenate([encoded_a , encoded_b],
                                        axis=-1)

predictions = Dense(1 , activation='sigmoid')(merged_vector)

model = Model(inputs=[tweet_a  ,tweet_b] , outputs=predictions)

model.compile(optimizer='rmsprop',
             loss='binary_crossentropy',
             metrics=['accuracy'])

model.fit([data_a , data_b] , labels , epochs=10)

In [ ]:
a = Input(shape=(140,256))

#output 32-dimensional vectors
lstm = LSTM(32)

encoded_a = lstm(a)

assert lstm.output == encoded_a

#lstm.output 层的输出
#lstm.output_shape 层输出的shape

In [ ]:
a = Input(shape=(140 , 256))
b = Input(shape=(140 , 256))

lstm = LSTM(32)
encoded_a = lstm(a)
encoded_b = lstm(b)

#lstm.output error

assert lstm.get_output_at(0) == encoded_a
assert lstm.get_output_at(1) == encoded_b


In [ ]:
from keras.layers import Conv2D
a = Input(shape=(32,32,3))
b = Input(shape=(64,64,3))

#one input
conv = Conv2D(16 , (3,3) , padding='same')
conved_a = conv(a)

assert conv.input_shape == (None , 32 , 32 , 3)

#multiple input
conved_b = conv(b)

assert conv.get_input_shape_at(0) == (None , 32 , 32 , 3)
assert conv.get_input_shape_at(1) == (None , 64 , 64 , 3)


In [ ]:
#google inception model

from keras.layers import Conv2D , MaxPooling2D , Input

input_img = Input(shape=(256,256,3))

tower_1 = Conv2D(64 , (1,1) , padding='same',
                activation='relu')(input_img)
tower_1 = Conv2D(64 , (3,3) , padding='same',
                activation='relu')(tower_1)

tower_2 = Conv2D(64 , (1,1) , padding='same',
                activation='relu')(input_img)
tower_1 = Conv2D(64 , (5,5) , padding='same',
                activation='relu')(tower_2)

tower_3 = MaxPooling2D((3,3) , strides=(1,1) , padding='same')(input_img)
tower_3 = Conv2D(64 , (1,1) , padding='same',
                 activation='relu')(tower_3)

output = keras.layers.concatenate([tower_1 , tower_2 , tower_3] , axis=1)

In [ ]:
#residual netword
